Exemplos *Classe TuboRet*
-----------------------
Este notebook apresenta exemplos de utilização da classe *TuboRet* que representa o perfil metálico do tipo *tubo retangular*. Estes exemplos tem por objetivo mostrar como utilizar a biblioteca **Metalpy**, apresentar algumas funcionalidades e servir como validação dos métodos implementados.

Os exemplos apresentados são exemplos já resolvidos obtidos de livros e manuais das seguintes referências:

[1] ARAÚJO, AFONSO et al. **Projeto de Estruturas de Edificações com perfis Tubulares em Aço.** Belo Horizonte, 2016.

[2] AMERICAN INSTITUTE OF STEEL CONSTRUCTION. AISC 360: **Companion to the AISC steel construction manual - Volume 1: Design Examples.** V15.1, Chicago. Illinois, 2015.

-----------------------------------------------------------------------------------------------------------------------------

### Exemplo 1

Este exemplo é o exemplo 3.7.4 de [1], trata do dimensionamento de uma viga biapoiada de 8 metros de vão, submetida a uma carga distribuida de 22kN/m, o perfil a ser dimensinado é do tipo retangular fabricado por conformação a frio. O aço utilizado no perfil é o aço ASTM A572 grau 50 (fy = 34.5kN/cm²). 
O perfil a ser verificado tem 300 mm de altura, 220 mm de largura e 6.3 mm de espessura, e será verificado ao momento fletor e esforço cortante.

Obs: O banco de dados do Metalpy não apresenta o perfil com as dimensões apresentadas, portanto será utiliazdo os perfil com as dimensões mais próximas TR300X200X6.4. Isso justifica a diferença dos resultados apresentado. 

In [1]:
from metalpy.perfis import TuboRet
from metalpy.material import Aco

import pandas as pd

#Definição do Aço com a propriedades em kN/cm²
A572 = Aco(20000, 0.3, 34.5, 45.0, 0.6)
L = 800 #cm

#Definindo um perfil do tipo tubo retangular
TR300X200 = TuboRet('TR300X200X6.4', A572, und='cm', norma='NBR8800')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [2]:
#Respostas do exercicio
r_Ag = 62.91 # Área bruta
r_Zx = 652 # Módulo plástico
r_Ix = 8245 # Momento de inércia
r_esb_mesa = 30.92 #Esbeltez da mesa
r_esb_alma = 43.62 #Esbeltez da Alma
r_indice_esbeltez = 88.59 # Indice de esbeltez
r_Mrdx = 18584 # Momento resistente
r_Vrdy = 652 # Força cortante resistente

#Respostas com Metalpy
mp_Ag = TR300X200.A
mp_Zx = TR300X200.Zx
mp_Ix = TR300X200.Ix
mp_esb_mesa = TR300X200.esb_mesa
mp_esb_alma =  TR300X200.esb_alma

#Obtendo o indice de esbeltez
Lbx = Lby = 800
iesb_x, iesb_y = TR300X200.indice_esbeltez(Lbx, Lby)

mp_indice_esbeltez = iesb_y

# Obtendo o momento fletor resistente em relação ao eixo X(maior inércia)
Lb = 800
Mrdx, dados_Mrd = TR300X200.Mrdx(Lb= Lb, Cb=1, data=True)
mp_Mrdx = Mrdx

# Obtendo a força cortante resistente
mp_Vrdy = TR300X200.Vrdy() 

columns = 'Referência Metalpy'.split()
index = ['Ag (cm²)', 'Zx (cm³)', 'Ix (cm4)', 'bf/2tf', 'h/tw', 'Lb/ry', 'Mrdx (kN.cm)', 'Vrd (kN)']
data = zip([r_Ag, r_Zx, r_Ix, r_esb_mesa, r_esb_alma, r_indice_esbeltez, r_Mrdx, r_Vrdy], [mp_Ag, mp_Zx, mp_Ix, mp_esb_mesa, mp_esb_alma, mp_indice_esbeltez, mp_Mrdx, mp_Vrdy])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Ag (cm²),62.91,58.700000
Zx (cm³),652.00,619.000000
Ix (cm4),8245.00,7732.000000
bf/2tf,30.92,28.250000
h/tw,43.62,43.875000
Lb/ry,88.59,95.018940
Mrdx (kN.cm),18584.00,18793.056796
Vrd (kN),652.00,676.370618


In [3]:
# Verificação
tol = 0.04 #tolerência de 4% de erro pelos fatos dos perfis não serem o mesmo

assert min(r_Mrdx, mp_Mrdx)/ max(r_Mrdx, mp_Mrdx) >= 1 - tol, 'Valor do momento resistente excede o tolerência'
assert min(r_Vrdy, mp_Vrdy)/ max(r_Vrdy, mp_Vrdy) >= 1 - tol, 'Valor da força cortante resistente excede o tolerência'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 2

Exemplo E.9 de [2], determinação da resistência a compressão de uma barra formada por um perfil tubo retangular HSS12X10X3/8 
(equivalente ao HSS304.8X254X9.5 em mmXmmXmm). O perfil apresenta um comprimento de flambagem de 16 ft (487,68 cm) e o aço é o ASTM A500 grau C.

* Ag = 14.6 in²
* t = 0.349 in
* rx = 4.61 in
* ry = 4.01 in
* b/t = 25.7
* h/tw = 31.4

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [4]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS304.8X254X9.5 em unidades de cm
tubo30x25 = TuboRet('HSS304.8X254X9.5', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [5]:
#Respostas do exercicio
r_A = 14.6 # Área total
r_t = 0.349 # Espessura
r_esb_mesa = 25.7 # Esbeltez da mesa
r_esb_alma =  31.4 # Esbeltez da alma
r_ry = 4.01 # Raio de giração em Y
r_rx = 4.61 # Raio de giração em X
r_Fe = 125 # Tensão critica de flambagem
r_Fcr = 42.3 # Tensão crítica
r_Ncrd = 556 # Força axial resistente de cálculo pela AISC

#Respostas com Metalpy
mp_A =  tubo30x25.A * cm_to_in ** 2
mp_t = tubo30x25.t * cm_to_in
mp_esb_mesa = tubo30x25.esb_mesa
mp_esb_alma = tubo30x25.esb_alma
mp_ry = tubo30x25.ry * cm_to_in
mp_rx = tubo30x25.rx * cm_to_in


# Calculando a força axial resistente de cálculo através do método Ncrd e obtendo os dados do cálculo
Lx = 16 * ft_to_cm
Ly = 16 * ft_to_cm
Lz = 1
Ncrd, dados = tubo30x25.Ncrd(Lx, Ly, 1, data=True)
mp_Ncrd = Ncrd * kN_to_kips
mp_Fe = dados.Fe * kN_cm2_to_ksi
mp_Fcr = dados.Fcr * kN_cm2_to_ksi

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 't (in)', 'bf/2tf', 'h/tw', 'ry (in)', 'rx (in)', 'Fe (ksi)', 'Fcr (ksi)', 'Ncrd (kips)']
data = zip([r_A, r_t, r_esb_mesa, r_esb_alma, r_ry, r_rx, r_Fe, r_Fcr, r_Ncrd], [ mp_A, mp_t, mp_esb_mesa, mp_esb_alma, mp_ry, mp_rx, mp_Fe, mp_Fcr, mp_Ncrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A (in²),14.600,14.623231
t (in),0.349,0.349084
bf/2tf,25.700,25.700000
h/tw,31.400,31.400000
ry (in),4.010,4.006363
rx (in),4.610,4.610687
Fe (ksi),125.000,124.433096
Fcr (ksi),42.300,42.259962
Ncrd (kips),556.000,555.798338


In [6]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Ncrd, mp_Ncrd)/ max(r_Ncrd, mp_Ncrd) >= 1 - tol, 'Valor da força axial resistente excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 3

Exemplo E.10 de [2], determinação da resistência a compressão de uma barra formada por um perfil tubo retangular de elementos esbeltos HSS12X8X3/16 (equivalente ao HSS304.8X203.2X4.8 em mmXmmXmm). O perfil apresenta um comprimento de flambagem de 24 ft (731,52 cm) e o aço é o ASTM A500 grau C.

* Ag = 6.76 in²
* t = 0.174 in
* rx = 4.56 in
* ry = 3.35 in
* b/t = 43
* h/tw = 66

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [7]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS304.8X203.2X4.8 em unidades de cm
tubo30x20 = TuboRet('HSS304.8X203.2X4.8', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [8]:
#Respostas do exercicio
r_A = 6.76 # Área total
r_t = 0.174 # Espessura
r_esb_mesa = 43 # Esbeltez da mesa
r_esb_alma =  66 # Esbeltez da alma
r_ry = 3.34 # Raio de giração em Y
r_rx = 4.56 # Raio de giração em X
r_Fe = 38.7 # Tensão critica de flambagem
r_Fcr = 29.1 # Tensão crítica
r_Ncrd = 151 # Força axial resistente de cálculo pela AISC

#Respostas com Metalpy
mp_A =  tubo30x20.A * cm_to_in ** 2
mp_t = tubo30x20.t * cm_to_in
mp_esb_mesa = tubo30x20.esb_mesa
mp_esb_alma = tubo30x20.esb_alma
mp_ry = tubo30x20.ry * cm_to_in
mp_rx = tubo30x20.rx * cm_to_in


# Calculando a força axial resistente de cálculo através do método Ncrd e obtendo os dados do cálculo
Lx = 24 * ft_to_cm
Ly = 24 * ft_to_cm
Lz = 1
Ncrd, dados = tubo30x20.Ncrd(Lx, Ly, 1, data=True)
mp_Ncrd = Ncrd * kN_to_kips
mp_Fe = dados.Fe * kN_cm2_to_ksi
mp_Fcr = dados.Fcr * kN_cm2_to_ksi

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 't (in)', 'bf/2tf', 'h/tw', 'ry (in)', 'rx (in)', 'Fe (ksi)', 'Fcr (ksi)', 'Ncrd (kips)']
data = zip([r_A, r_t, r_esb_mesa, r_esb_alma, r_ry, r_rx, r_Fe, r_Fcr, r_Ncrd], [ mp_A, mp_t, mp_esb_mesa, mp_esb_alma, mp_ry, mp_rx, mp_Fe, mp_Fcr, mp_Ncrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A (in²),6.760,6.768290
t (in),0.174,0.174148
bf/2tf,43.000,43.000000
h/tw,66.000,66.000000
ry (in),3.340,3.348947
rx (in),4.560,4.556037
Fe (ksi),38.700,38.642872
Fcr (ksi),29.100,29.091931
Ncrd (kips),151.000,151.346334


In [9]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Ncrd, mp_Ncrd)/ max(r_Ncrd, mp_Ncrd) >= 1 - tol, 'Valor da força axial resistente excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 4

Exemplo F.6 de [2], determinação do momento fletor resistente de uma viga formada por um perfil tubo retangular HSS 3.5X3.5X1/18 (equivalente ao HSS88.9X88.9X3.2 em mmXmmXmm). O perfil está contido em todo o seu comprimento e é formado por aço ASTM A500 grau C.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [10]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
ksi_to_kN_cm2 = 1 / 1.4504
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm) 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS retangular
tubo90x90 = TuboRet('HSS88.9X88.9X3.2', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [11]:
#Respostas do exercicio
r_Mrd = 7.21 # Momento fletor resistente 

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 0
Cb = 1
Mrd= tubo90x90.Mrdx(Lb = Lb, Cb = Cb)
mp_Mrd = Mrd * kNcm_to_kipft

columns = 'Referência Metalpy'.split()
index = ['Mrd (kip.ft)']
data = zip([r_Mrd], [ mp_Mrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Mrd (kip.ft),7.21,7.205075


In [12]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd, mp_Mrd)/ max(r_Mrd, mp_Mrd) >= 1 - tol, 'Valor do momento fletor resistente excede tolerância'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 5

Exemplo F.7 de [2], determinação do momento fletor resistente de uma viga formada por um perfil tubo retangular de mesas não compactas HSS 10X6X3/16 (equivalente ao HSS254X152.4X4.8 em mmXmmXmm). O perfil está contido apenas nas extremidades de apoio e é formado por aço ASTM A500 grau C.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [13]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS retangular
tubo25x15 = TuboRet('HSS254X152.4X4.8', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [14]:
#Respostas do exercicio
r_Ag = 5.37 # Area total
r_Zx = 18 # Módulo plástico
r_Sx = 14.9 # Módulo elástico
r_ry = 2.52 # Raio de giração em Y
r_J= 73.8 # Inércia a torção
r_esb_mesa = 31.5 # Esbeltez da mesa
r_esb_alma = 54.5 # Esbeltez da alma
r_Lp = 210 # Comprimento limite plástico
r_Lr = 5580 # Comprimento limite para o inicio de escoamento
r_Mrd= 59.8 # Momento resistente de cálculo

#Respostas com Metalpy
mp_Ag = tubo25x15.A * cm_to_in ** 2
mp_Zx = tubo25x15.Zx * cm_to_in ** 3
mp_Sx = tubo25x15.Wx * cm_to_in ** 3
mp_ry =  tubo25x15.ry * cm_to_in
mp_J = tubo25x15.J * cm_to_in ** 4
mp_esb_mesa = tubo25x15.esb_mesa
mp_esb_alma = tubo25x15.esb_alma

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 21 * ft_to_cm
Cb = 1.14
Mrd, dados_Mrd= tubo25x15.Mrdx(Lb = Lb, Cb = Cb, data=True)
mp_Mrd = Mrd * kNcm_to_kipft
mp_Lp = dados_Mrd.FLT.Lp * cm_to_in
mp_Lr = dados_Mrd.FLT.Lr * cm_to_in

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 'Zx (in³)', 'Sx (in³)', 'ry (in)', 'J (in³)', 'Lp (in)', 'Lr (in)', 'Mrd (kip.ft)']
data = zip([r_Ag, r_Zx, r_ry, r_Sx, r_J, r_Lp, r_Lr, r_Mrd], [ mp_Ag, mp_Zx, mp_ry, mp_Sx, mp_J, mp_Lp, mp_Lr, mp_Mrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A (in²),5.37,5.371166
Zx (in³),18.00,18.043080
Sx (in³),2.52,2.524076
ry (in),14.90,14.923768
J (in³),73.80,73.981522
Lp (in),210.00,210.261090
Lr (in),5580.00,5587.013293
Mrd (kip.ft),59.80,59.744426


In [15]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd, mp_Mrd)/ max(r_Mrd, mp_Mrd) >= 1 - tol, 'Valor do momento fletor resistente excede tolerância'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 6

Exemplo F.8 de [2], determinação do momento fletor resistente de uma viga formada por um perfil tubo retangular de mesas esbeltaz HSS 8X8X3/16 (equivalente ao HSS203.2X203.2X4.8 em mmXmmXmm). O perfil está contido apenas nas extremidades de apoio e é formado por aço ASTM A500 grau C.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [16]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS retangular
tubo20x20 = TuboRet('HSS203.2X203.2X4.8', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [17]:
#Respostas do exercicio
r_Ix = 54.4 #Momento de inércia
r_Zx = 15.7 # Módulo plástico
r_Sx = 13.6 # Módulo elástico
r_B = 8 # Raio de giração em Y
r_H= 8 # Inércia a torção
r_esb_mesa = 43 # Esbeltez da mesa
r_esb_alma = 43 # Esbeltez da alma
r_Mrd= 45.4 # Momento resistente de cálculo

#Respostas com Metalpy
mp_Ix = tubo20x20.Ix * cm_to_in ** 4
mp_Zx = tubo20x20.Zx * cm_to_in ** 3
mp_Sx = tubo20x20.Wx * cm_to_in ** 3
mp_B =  tubo20x20.b * cm_to_in
mp_H = tubo20x20.h * cm_to_in
mp_esb_mesa = tubo20x20.esb_mesa
mp_esb_alma = tubo20x20.esb_alma

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 21 * ft_to_cm
Cb = 1.14
Mrd, dados_Mrd= tubo20x20.Mrdx(Lb = Lb, Cb = Cb, data=True)
mp_Mrd = Mrd * kNcm_to_kipft

columns = 'Referência Metalpy'.split()
index = ['Ix (in4)', 'Zx (in³)', 'Sx (in³)', 'B (in)', 'H (in)', 'b/t', 'h/t', 'Mrd (kip.ft)']
data = zip([r_Ix, r_Zx, r_Sx, r_B, r_H, r_esb_mesa, r_esb_alma, r_Mrd], [ mp_Ix, mp_Zx, mp_Sx, mp_B, mp_H, mp_esb_mesa, mp_esb_alma, mp_Mrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Ix (in4),54.4,54.461967
Zx (in³),15.7,15.718887
Sx (in³),13.6,13.639345
B (in),8.0,7.998200
H (in),8.0,7.998200
b/t,43.0,43.000000
h/t,43.0,43.000000
Mrd (kip.ft),45.4,45.108318


In [18]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd, mp_Mrd)/ max(r_Mrd, mp_Mrd) >= 1 - tol, 'Valor do momento fletor resistente excede tolerância'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 7

Exemplo G.4 de [2], determinação da força cortante resistente de uma viga formada por um perfil tubo retangular HSS 6X4X3/8 (equivalente ao HSS152.4X101.6X9.5 em mmXmmXmm). A viga é formada por aço ASTM A500 grau C.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [19]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS retangular
tubo15x10 = TuboRet('HSS152.4X101.6X9.5', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [20]:
#Respostas do exercicio
r_B = 4 # Raio de giração em Y
r_H= 6 # Inércia a torção
r_t = 0.349 # Espessura do perfil
r_esb_alma = 14.2 # Esbeltez da alma
r_Aw = 3.46 # Area efetiva de cisalhamento
r_Vrd= 93.6# Momento resistente de cálculo

#Respostas com Metalpy
mp_B =  tubo15x10.b * cm_to_in
mp_H = tubo15x10.h * cm_to_in
mp_t = tubo15x10.t * cm_to_in
mp_esb_alma = tubo15x10.esb_alma
mp_Aw = tubo15x10.Awy * cm_to_in ** 2


# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Vrd, dados = tubo15x10.Vrdy(data=True)
mp_Vrd = Vrd * kN_to_kips

columns = 'Referência Metalpy'.split()
index = ['B (in)', 'H (in)', 't (in)', 'h/t', 'Vrd (kips)']
data = zip([r_B, r_H, r_t, r_esb_alma, r_Vrd], [ mp_B, mp_H, mp_t, mp_esb_alma, mp_Vrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
B (in),4.000,4.018800
H (in),6.000,5.988800
t (in),0.349,0.349084
h/t,14.200,14.200000
Vrd (kips),93.600,93.377937


In [21]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Vrd, mp_Vrd)/ max(r_Vrd, mp_Vrd) >= 1 - tol, 'Valor da força cortante resistente excede o tolerência'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 8

Exemplo H.5 de [2], determinação da força resistência a torção de uma barra formada por um perfil tubo retangular HSS 6X4X1/4 (equivalente ao HSS152.4X101.6X6.4 em mmXmmXmm). A barra é formada por aço ASTM A500 grau C.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [22]:
from metalpy.perfis import TuboRet
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A500 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil HSS retangular
tubo15x10 = TuboRet('HSS152.4X101.6X6.4', A500, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [23]:
#Respostas do exercicio
r_t = 0.233 # Espessura do perfil
r_esb_alma = 22.8 # Esbeltez da alma
r_esb_mesa = 14.2 # Esbeltez da mesa
r_C= 10.1# Momento resistente a torção
r_Trd = 273

#Respostas com Metalpy
mp_t = tubo15x10.t * cm_to_in
mp_esb_alma = tubo15x10.esb_alma
mp_esb_mesa = tubo15x10.esb_mesa
mp_C = tubo15x10.Wt * cm_to_in ** 3


# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Trd, dados = tubo15x10.Trd(L=0, data=True)
mp_Trd = Trd * kNcm_to_kipin 

columns = 'Referência Metalpy'.split()
index = ['t (in)', 'h/t', 'b/t', 'C (in³)', 'Trd (kip.in)']
data = zip([r_t, r_esb_alma, r_esb_mesa, r_C, r_Trd], [ mp_t, mp_esb_alma, mp_esb_mesa, mp_C, mp_Trd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
t (in),0.233,0.233248
h/t,22.800,22.800000
b/t,14.200,14.200000
C (in³),10.100,10.153055
Trd (kip.in),273.000,273.944636


In [24]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Trd, mp_Trd)/ max(r_Trd, mp_Trd) >= 1 - tol, 'Valor da torção resistente excede o tolerência'

-------------------------------------------------------------------------------------------------------------------------
### Exemplo 9

Exemplo 3.7.8 de [1]. Verificação de pilar flexo-comprimido  formado por um perfil quadrado TQ 220X220X16 de aço VMB 350 ( $fy = 350MPa$) que apresenta um comprimento de 4m. O pilar está submetido a um força de compressão de $N_{csd} = 296,46 kN$ e um momento em relação ao eixo X (horizontal) de $M_{xsd} = 70.92 kN.m$, solicitações de cálculo. A imagem abaixo apresenta as propriedades do perfil.


 <img src="img\\EX9_Tubo_Ret.png">

 A verificação será realizada de acordo com a *NBR8800:2008*

In [25]:
from metalpy.perfis import TuboRet
from metalpy.material import Aco

import pandas as pd

#Definição do Aço com a propriedades em kN/cm²
VMB350 = Aco(20000, 0.3, 35, 45.0, 0.6)
L = 400 #cm

#Definindo um perfil do tipo tubo retangular TQ 220X220X16 com as propriedades em cm
TQ220X16 = TuboRet('TQ220X220X16', VMB350, und='cm', norma='NBR8800')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [26]:
#Respostas do exercicio
r_A = 129 # Area total
r_Ix = r_Iy = 7812 # Momento de inércia
r_Wx = r_Wy = 710 # Módulo elástico
r_Zx = 881 # Módulo plástico
r_rx = r_ry = 7.78 # Raio de giração
r_J = 13970 # Inércia a torção
r_esb_mesa = r_esb_alma = 10.75 # Esbeltez dos elementos do perfil
r_Q = 1 # Fator de redução da força de compressão devido a flambagem local
r_Nex = r_Ney = 9638 # Força de flambagem por flexão em relação aos eixos X e Y
r_Nez = 889000 # Força de flambagem por torção
r_ier = 0.68 # Indice de esbeltez reduzido
r_Chi = 0.93 # Fator de redução devido a flambagem global
r_Ncrd = 3817 #Força de compressão resistente de cálculo
r_Mrdx = 28032 # Momento resistente de cálculo
r_verif = 0.29

#Respostas com Metalpy
mp_A = TQ220X16.A
mp_Ix = TQ220X16.Ix
mp_Iy = TQ220X16.Iy
mp_Wx = TQ220X16.Wx
mp_Wy = TQ220X16.Wy
mp_Zx = TQ220X16.Zx
mp_rx = TQ220X16.rx
mp_ry = TQ220X16.ry
mp_J = TQ220X16.J
mp_esb_mesa = TQ220X16.esb_mesa
mp_esb_alma = TQ220X16.esb_alma

#Calculando as cargas criticas de flambagema través do método par_estabilidade
Lx = 400
Ly = 400
Lz = 400

par_estabilidade = TQ220X16.par_estabilidade(Lx, Ly, Lz)

mp_Nex = par_estabilidade.Nex
mp_Ney = par_estabilidade.Ney
mp_Nez = par_estabilidade.Nez

# Calculando a força de compressão resistente de cálculo através do método Ncrd e obtendo os dados do cálculo
mp_Ncrd, dados_Ncrd = TQ220X16.Ncrd(Lx, Ly, Lz , data=True)

mp_Q = dados_Ncrd.Q
mp_ier = dados_Ncrd.ier
mp_Chi = dados_Ncrd.Chi

#Calculando o momento resistente de cálculo

mp_Mrdx = TQ220X16.Mrdx()

# Verificação a flexo-compressão
Ncsd = -296.46
Mxsd = 7092
Mysd = 0

mp_verif = TQ220X16.verif_NM(Ncsd, Mxsd, Mysd, klx=Lx, kly=Ly, klz=Lz)

columns = 'Referência Metalpy'.split()
index = ['A(cm²)', 'Ix(cm4)', 'Iy(cm4)', 'Wx(cm³)', 'Wy(cm³)', 'Zx(cm³)', 'rx(cm)', 'ry(cm)', 'J(cm³)',  'bf/2tf', 'h/tw', 'Q', 'Nex (kN)', 'Ney(kN)', 'Nez(kN)', 'lambda_0', 'Chi', 'Ncrd(kN)', 'Mrdx(kN.cm)', 'NM']
data = zip([r_A, r_Ix, r_Iy, r_Wx, r_Wy, r_Zx, r_rx, r_ry, r_J, r_esb_mesa, r_esb_alma, r_Q,  r_Nex, r_Ney, r_Nez, r_ier, r_Chi, r_Ncrd, r_Mrdx, r_verif], [mp_A, mp_Ix, mp_Iy, mp_Wx, mp_Wy, mp_Zx, mp_rx, mp_ry, mp_J, mp_esb_mesa, mp_esb_alma, mp_Q,  mp_Nex, mp_Ney, mp_Nez, mp_ier, mp_Chi, mp_Ncrd, mp_Mrdx, mp_verif])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A(cm²),129.00,129.000000
Ix(cm4),7812.00,7812.000000
Iy(cm4),7812.00,7812.000000
Wx(cm³),710.00,710.000000
Wy(cm³),710.00,710.000000
Zx(cm³),881.00,881.000000
rx(cm),7.78,7.781911
ry(cm),7.78,7.781911
J(cm³),13970.00,13970.000000
bf/2tf,10.75,10.750000
